In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets
from PIL import Image
import pandas as pd
import os
from torch.optim.lr_scheduler import StepLR
import json
import matplotlib.pyplot as plt

In [2]:
class Bottleneck(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channels)
        self.conv3 = nn.Conv2d(mid_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        residual = self.downsample(residual)
        out += residual
        out = self.relu(out)
        return out

In [3]:

class Net(nn.Module):
    def __init__(self, num_classes=5):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            Bottleneck(96, 32, 128),
            
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            Bottleneck(256, 64, 512),
            
            nn.Conv2d(512, 768, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            Bottleneck(768, 128, 768),
            
            nn.Conv2d(768, 1024, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),

            Bottleneck(1024, 128, 1280), 
            
            # nn.Conv2d(1024, 768, kernel_size=3, stride=1, padding=1),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=3, stride=2),
            
            # Bottleneck(768, 256, 1024),
            
            nn.Conv2d(1280, 1536, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            Bottleneck(1536, 320, 2048),
            
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(2048, 1280),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1280, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

net = Net(num_classes=5)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
class CassavaDataset(Dataset):
    def __init__(self, csv_f, img_folder, transform, s_num=6000):
        self.img_labels = pd.read_csv(csv_f)[:s_num]
        self.img_folder = img_folder
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_name = str(self.img_labels.iloc[idx, 0])
        label = self.img_labels.iloc[idx, 1]
        img_path = os.path.join(self.img_folder, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


pre = transforms.Compose([transforms.RandomResizedCrop(224),
                          transforms.RandomHorizontalFlip(),
                          transforms.RandomRotation(15),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                          ])

csv_f = '../data/cassava-leaf-disease-classification/train.csv'
img_folder = '../data/cassava-leaf-disease-classification/train_images'
data = CassavaDataset(csv_f, img_folder, transform=pre, s_num=6000)

train_size = int(0.9 * len(data))
train_dataset, test_dataset = random_split(data, [train_size, (len(data) - train_size)])

batch_size = 32

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
num_classes = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#net = Net( num_classes= 5)
net.to(device)
loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

t_steps = len(trainloader)
test_num = len(test_dataset)

for epoch in range( 32 ):
    net.train()
    running_loss = 0.0
    for step, data in enumerate(trainloader, start=0):
        imgs, labels = data
        rate = ((step + 1) / t_steps)
        optimizer.zero_grad()
        outputs = net(imgs.to(device))
        loss = loss_f(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        a = '*' * int(rate * 50)
        b = '.' * int((1 - rate) * 50)
        print('\r train loss: {:^3.0f}%[ {} -> {} ]{:.3f}'.format(int(rate * 100), a, b, loss), end='')
    print()

    net.eval()
    r = 0.0
    best_R = 0.0
    with torch.no_grad():
        for data_test in testloader:
            t_img, t_lbl = data_test
            outputs = net(t_img.to(device))
            pred_y = torch.max(outputs, dim=1)[1]
            r += (pred_y == t_lbl.to(device)).sum().item()
        acc_R = r / test_num
        if acc_R > best_R:
            best_R = acc_R
            torch.save(net.state_dict(), '../path/Net_best.pth')
        print('[epoch %d] train_loss: %.3f    test_accuracy: %.3f' % (epoch + 1, running_loss, acc_R))

    print('finished')

 train loss: 100%[ ************************************************** ->  ]0.872
[epoch 1] train_loss: 202.720    test_accuracy: 0.547
finished
 train loss: 100%[ ************************************************** ->  ]1.179
[epoch 2] train_loss: 189.210    test_accuracy: 0.590
finished
 train loss: 100%[ ************************************************** ->  ]0.899
[epoch 3] train_loss: 182.764    test_accuracy: 0.583
finished
 train loss: 15 %[ ******* -> .......................................... ]0.971

KeyboardInterrupt: 